In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import logging
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 3)


In [3]:
url = "https://hudoc.echr.coe.int/eng?i=001-233206"
driver.get(url)

In [4]:
elements = driver.find_elements(By.CSS_SELECTOR, 'p, h1, h2, h3, h4, h5, h6')

#elements = driver.find_elements(By.XPATH, "//*[not(self::script or self::style or self::noscript)][string-length(normalize-space(text())) > 0]")

#elements = driver.find_elements(By.CSS_SELECTOR, 
#    'p, h1, h2, h3, h4, h5, h6, div, span, article, section, main, li, td, th, strong, em, b, i, label, a, blockquote, pre, code, caption')

In [5]:
tabs = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, ".nav-tabs li")
            ))
tabs[1].click()
elements = driver.find_elements(By.CSS_SELECTOR, '.noticefield')
for idx, elem in enumerate(elements):
    line = elem.text.strip()
    if not line:
        continue
    else:
        print(f"{idx}: {line}")

0: Originating Body
Court (Grand Chamber)
1: Document Type
Judgment (Merits and Just Satisfaction)
2: Title
CASE OF VEREIN KLIMASENIORINNEN SCHWEIZ AND OTHERS v. SWITZERLAND
3: App. No(s).
53600/20
4: Importance Level
Key cases
5: Respondent State(s)
Switzerland
6: Judgment Date
09/04/2024
7: Applicability
Art. 6 applicable
Art. 8 applicable
8: Conclusion(s)
Preliminary objection dismissed (Art. 34) Individual applications
(Art. 34) Locus standi
Preliminary objection dismissed (Art. 35) Admissibility criteria
(Art 35-1) Four-month period (former six-month)
Preliminary objection allowed (Art. 34) Individual applications
(Art. 34) Victim
Remainder inadmissible (Art. 35) Admissibility criteria
(Art. 35-3-a) Ratione materiae
(Art. 35-3-a) Ratione personae
Violation of Article 8 - Right to respect for private and family life (Article 8 - Positive obligations
more…
9: Article(s)
34
35
35-1
35-3-a
8
8-1
6
6-1
46
46-2
more…
10: Separate Opinion(s)
Yes
11: Domestic Law
Federal Act on Climate Pr

In [13]:
metadata_fields = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, ".notice-metadata tr")
))


TimeoutException: Message: 


In [ ]:
    def scrape_metadata(self):
        """Scrape metadata from the metadata tab."""
        try:
            metadata = {}
            # Switch to metadata tab
            if self.switch_to_tab("Case Details"):
                # Wait for metadata fields to be present
                metadata_fields = self.wait.until(EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, ".notice-metadata tr")
                ))
                
                for field in metadata_fields:
                    try:
                        key = field.find_element(By.CSS_SELECTOR, "th").text.strip()
                        value = field.find_element(By.CSS_SELECTOR, "td").text.strip()
                        if key and value:
                            metadata[key] = value
                    except:
                        continue
            
            return metadata
        except Exception as e:
            print(f"Error scraping metadata: {str(e)}")
            return {}


In [67]:
start_after_index = 0
f = False
for idx, elem in enumerate(elements):
    line = elem.text.strip()
    if not line:
        continue
    if line.startswith("TABLE OF CONTENTS"):
        print(idx)
        f = True
    if ("CONCLUSION" in line) or ('Conclusion' in line):
        print(idx)
        start_after_index = idx
        f = False
        break
    if f:
        print(line)

start_after_index

64
TABLE OF CONTENTS
PROCEDURE
THE FACTS
I. THE CIRCUMSTANCES OF THE CASE
A. The applicants’ particular situation
1. The first applicant
2. The second to fifth applicants
(a) The second applicant
(b) The third applicant
(c) The fourth applicant
(d) The fifth applicant
B. Proceedings instituted by the applicants
1. The applicants’ requests to the authorities
2. Proceedings in the Federal Administrative Court
(a) The applicants’ appeal
(b) The FAC’s decision
3. Proceedings in the Federal Supreme Court
(a) The applicants’ appeal
(b) The FSC’s decision
II. FACTS CONCERNING CLIMATE CHANGE
A. Submissions by the applicants
1. General observations on climate change
2. The situation in Switzerland
3. Measures taken by the Swiss authorities
B. Submissions by the Government
1. The first phase
2. The second phase
C. Facts in relation to climate change emerging from the material available to the Court
RELEVANT LEGAL FRAMEWORK AND PRACTICE
I. DOMESTIC LEGAL FRAMEWORK
A. Constitution
B. Federal Act o

240

In [68]:

facts = []
flag=False
for elem in elements[start_after_index:]:
    line = elem.text.strip()
    if not line:
        continue
    if line.startswith("THE FACTS"):
        flag=True
        facts.append(line)
    elif line.startswith("THE LAW") or line.startswith("RELEVANT LEGAL FRAMEWORK AND PRACTICE"):
        flag=False
        break
    if flag:
        facts.append(line)

In [69]:
facts

['THE FACTS',
 'THE FACTS',
 '10.  The first applicant – Verein KlimaSeniorinnen Schweiz – is a non‑profit association established under Swiss law (“the applicant association”). According to its Statute, the applicant association was established to promote and implement effective climate protection on behalf of its members. The members of the association are women living in Switzerland, the majority of whom are over the age of 70. The applicant association is committed to reducing greenhouse gas (“GHG”) emissions in Switzerland and their effects on global warming. The activity of the applicant association is stated to be in the interests of not only its members, but also of the general public and future generations, through effective climate protection. The applicant association pursues its purpose in particular through the provision of information, including educational activities, and by taking legal action in the interests of its members with regard to the effects of climate change.

In [22]:
for elem in elements:
    line = elem.text.strip()
    if not line:
        continue
    else:
        print(line)

NARROW YOUR SEARCH
DOCUMENT COLLECTIONS
MORE HUDOC SITES
FILTERS
LANGUAGE
IMPORTANCE
STATE
ARTICLE
NON-VIOLATION
VIOLATION
KEYWORDS
MORE FILTERS
GRAND CHAMBER
CASE OF DUARTE AGOSTINHO AND OTHERS
AGAINST PORTUGAL AND 32 OTHERS
(Application no. 39371/20)
DECISION
STRASBOURG
9 April 2024
TABLE OF CONTENTS
PROCEDURE
THE FACTS
I. FACTS SUBMITTED BY THE APPLICANTS
A. Facts submitted in the application form
1. Applicants nos. 1-3
2. Applicant no. 4
3. Applicants nos. 5-6
B. Facts submitted to the Grand Chamber
II. FACTS SUBMITTED BY THE RESPONDENT GOVERNMENTS
RELEVANT LEGAL FRAMEWORK AND PRACTICE
I. DOMESTIC LEGAL FRAMEWORK OF PORTUGAL
A. Relevant domestic law
1. The Constitution
2. Law no. 83/95 of 31 August 1995
3. Civil Code
4. Code of Civil Procedure
5. Law no. 19/2014 of 14 April 2014
6. Law no. 67/2007 of 31 December 2007
7. The Code of Administrative Courts Procedure
8. Law no. 35/98 of 18 July 1998
9. Decree-Law no. 147/2008 of 20 July 2008
10. Law no. 98/2021 of 31 December 2021
11. 